<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">INFO</h1>

In this script, we create a class that will structure the unit tests for the `DFS` player. \
We choose to use the `unittest` library. \
Then, we run them to ensure that all methods developed work as expected.

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">IMPORTS</h1>

In [1]:
# External imports
from typing import *
from numbers import *
import unittest
import sys
import os

# Add needed directories to the path
sys.path.append(os.path.join("..", "players"))

# PyRat imports
from GreedyEachTurn import GreedyEachTurn
from pyrat import BigHolesRandomMaze, Graph,GameState


<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">DEFINE THE TESTS</h1>

The `unittest` library requires the creation of a class that extends `unittest.TestCase`. \
For each method to test, we need to define a method in the test class. \
Each of these test methods should call the tested method with various inputs to check that produced outputs match expected ones.

In [2]:
import unittest
from pyrat import Graph, Action, PlayerSkin
from GreedyEachTurn import GreedyEachTurn

class GreedyEachTurnTests(unittest.TestCase):
    def test_dijkstra(self):
        """
        I/O:
        I: Aucun argument direct (on crée un graphe simple)
        O: None (assertions)

        Explication:
        Teste la méthode dijkstra sur un graphe simple à 4 sommets.
        Vérifie que les distances et les précédents sont corrects.
        """
        graph = Graph()
        for i in range(4):
            graph.add_vertex(i)
        graph.add_edge(0, 1, 1)
        graph.add_edge(1, 2, 2)
        graph.add_edge(2, 3, 3)

        player = GreedyEachTurn(skin=PlayerSkin.RAT)
        source = 0
        distances, previous = player.dijkstra(graph, source)

        expected_distances = {0: 0.0, 1: 1.0, 2: 3.0, 3: 6.0}
        expected_previous = {0: None, 1: 0, 2: 1, 3: 2}

        self.assertEqual(distances, expected_distances)
        self.assertEqual(previous, expected_previous)

    def test_find_route(self):
        """
        I/O:
        I: Aucun argument direct (précédents fixés)
        O: None (assertions)

        Explication:
        Teste find_route avec un dictionnaire de précédents.
        Vérifie que le chemin reconstruit est celui attendu.
        """
        player = GreedyEachTurn(skin=PlayerSkin.RAT)
        previous = {0: None, 1: 0, 2: 1, 3: 2}
        start, target = 0, 3
        expected_path = [1, 2, 3]

        path = player.find_route(previous, start, target)
        self.assertEqual(path, expected_path)

    def test_get_action(self):
        """
        I/O:
        I: Aucun argument direct (MockMaze)
        O: None (assertions)

        Explication:
        Teste get_action pour s'assurer que les directions renvoyées
        correspondent aux attentes en fonction des positions source et cible.
        """
        player = GreedyEachTurn(skin=PlayerSkin.RAT)
        class MockMaze:
            def i_to_rc(self, index):
                return divmod(index, 10)
        maze = MockMaze()

        self.assertEqual(player.get_action(0, 1, maze), Action.EAST)
        self.assertEqual(player.get_action(0, 10, maze), Action.SOUTH)
        self.assertEqual(player.get_action(10, 0, maze), Action.NORTH)
        self.assertEqual(player.get_action(1, 0, maze), Action.WEST)
        self.assertEqual(player.get_action(0, 0, maze), Action.NOTHING)

    def test_turn_recalculates_each_turn(self):
        """
        I/O:
        I: Aucun argument direct (MockMaze, MockGameState)
        O: None (assertions)

        Explication:
        Teste que le joueur recalculent la cible à chaque tour.
        On simule la disparition d'un fromage entre deux tours et on vérifie
        que le joueur change correctement de cible.
        """
        player = GreedyEachTurn(skin=PlayerSkin.RAT)
        player.position = 0

        class MockMaze:
            def i_to_rc(self, index):
                return divmod(index, 10)
            vertices = [0, 1, 2, 3]
            def get_neighbors(self, index):
                neighbors = {
                    0: [1],
                    1: [0, 2],
                    2: [1, 3],
                    3: [2]
                }
                return neighbors.get(index, [])
            def get_weight(self, u, v):
                return 1

        maze = MockMaze()

        class MockGameState:
            def __init__(self):
                self.player_locations = {player.name: player.position}
                self.cheese = [2, 3]

        game_state = MockGameState()

        # Premier tour : cible le fromage 2
        action = player.turn(maze, game_state)
        self.assertEqual(action, Action.EAST)

        # On simule que le joueur se déplace sur 1
        player.position = 1
        game_state.player_locations[player.name] = player.position

        # L'adversaire prend le fromage 2
        game_state.cheese.remove(2)

        # Tour suivant : le joueur doit viser le fromage 3
        action = player.turn(maze, game_state)
        self.assertEqual(action, Action.EAST)

    def test_no_cheeses_left(self):
        """
        I/O:
        I: Aucun argument direct (MockMaze, MockGameState)
        O: None (assertions)

        Explication:
        Teste le comportement quand il n'y a plus de fromages.
        Le joueur ne doit pas bouger et retourner NOTHING.
        """
        player = GreedyEachTurn(skin=PlayerSkin.RAT)
        player.position = 0

        class MockMaze:
            def i_to_rc(self, index):
                return divmod(index, 10)
            vertices = [0]
            def get_neighbors(self, index):
                return []
            def get_weight(self, u, v):
                return 1

        maze = MockMaze()

        class MockGameState:
            def __init__(self):
                self.player_locations = {player.name: player.position}
                self.cheese = []

        game_state = MockGameState()

        action = player.turn(maze, game_state)
        self.assertEqual(action, Action.NOTHING)


<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">RUN THE TESTS</h1>
           
When calling `unittest.main()`, all methods in the test class above will be run.

In [3]:
# Run all tests
_ = unittest.main(argv=[""], verbosity=2, exit=False)

test_dijkstra (__main__.GreedyEachTurnTests.test_dijkstra)
I/O: ... ok
test_find_route (__main__.GreedyEachTurnTests.test_find_route)
I/O: ... ok
test_get_action (__main__.GreedyEachTurnTests.test_get_action)
I/O: ... ok
test_no_cheeses_left (__main__.GreedyEachTurnTests.test_no_cheeses_left)
I/O: ... ok
test_turn_recalculates_each_turn (__main__.GreedyEachTurnTests.test_turn_recalculates_each_turn)
I/O: ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.013s

OK
